In [ ]:
# theme: flatly, lumen, sandstone, yeti
import polars as pl
from great_tables import GT, _data_color, md, nanoplot_options
from IPython.display import Markdown

In [ ]:
df = pl.read_parquet(
    f"../data/arrivals_polars.parquet"
).sort("caltrans_district", "n_trips", descending=[False, False])

In [ ]:
all_districts = sorted(df.get_column("caltrans_district").unique())

In [ ]:
def get_hex(color_name: str) -> str:
    """
    Since some of the color names don't pull the hex code, 
    we'll grab it here.
    """
    return _data_color.constants.COLOR_NAME_TO_HEX[color_name]

nano_options = nanoplot_options(
    data_point_radius=0,
    data_point_stroke_color="none",
    data_point_stroke_width=0,
    data_line_type="curved",
    data_line_stroke_color=get_hex("steelblue4"),
    data_line_stroke_width=8,
    data_area_fill_color=get_hex("steelblue4"),
    #vertical_guide_stroke_color=None,
    show_y_axis_guide=True,
    #show_vertical_guides=False,
    interactive_data_values = True,
    #reference_line_color=get_hex("salmon1"),
    show_reference_line=False
)

In [ ]:
# https://quarto.org/docs/output-formats/html-themes.html
# https://github.com/posit-dev/great-tables/blob/main/great_tables/_data_color/constants.py
def make_plot(df):
    """
    Make table for each district.
    """
    # equivalent of df.iloc
    district_name = df.item(0, "caltrans_district")
    
    table = (GT(df)
    .fmt_integer(
          columns = ["n_trips"]
     ).data_color(
        columns=["n_trips"],
        palette=["white", get_hex("mediumseagreen")],
        na_color="lightgray"
     ).fmt_nanoplot(
         columns = "weekday_arrivals", 
         plot_type="line",
         reference_line="mean",
         options = nano_options
     ).fmt_nanoplot(
         columns = "weekend_arrivals", 
         plot_type="line",
         reference_line="mean",
         options = nano_options
     ).cols_label(
         name = "Operator",
         n_trips = "Weekly Trips",
         weekday_arrivals = "Weekday Arrivals",
         weekend_arrivals = "Weekend Arrivals",
     ).sub_missing(
         columns = ["weekday_arrivals", "weekend_arrivals"],
         missing_text = ""
     ).tab_header(
         title = f"Stop Arrivals by Hour",
         #subtitle = "weekday vs weekend totals"
     )#.tab_source_note(
      #   source_note = "GTFS scheduled stop_times.txt"
     #).tab_source_note(
      #  source_note = md(
      #      "[GTFS specification](https://gtfs.org/schedule/reference/#stop_timestxt)"
      #  )
     #)
    .tab_options(
        container_width = "100%",
        table_background_color="white",
        table_body_hlines_style="none",
        table_body_vlines_style="none",
        heading_background_color="white",
        column_labels_background_color="white",
        row_group_background_color="white",
        stub_background_color="white",
        source_notes_background_color="white"
     )
    .cols_hide("caltrans_district")
    )
    return table

Have you ever been curious about transit service patterns in California? What would hourly service patterns tell us about the types of riders the operator is oriented to serve? 

Service geared towards commuters have pronounced camel peaks during the AM and PM peak commuting hours. Service geared towards travelers would provide mostly consistent service throughout the day. After all, travelers passing through the airport expect to catch a shuttle, no matter the time-of-day.

We can use GTFS scheduled stop times to count the number of arrivals for each hour of the day and look at how weekday and weekend service differ by operators. Within each Caltrans District, operators are sorted according to the number of weekly trip volume. **A `great_tables` nanoplot is a great way to convey, at a glance, the operator's hourly service profile while situating the operator's service in a regional context.** 

## Service Patterns

### Higher Peak Service with Steady Midday Service
* This is a common service pattern. There are slight camel peaks during the AM and PM peak periods. Service may drop slightly during early morning, midday, and evening periods, but there is still considerable service. Typically there is no owl service.
* Clearer examples by district: [Redwood Coast (District 1)](#eureka), [Roseville and Yolobus (District 3)](#marysville), [PresidiGo (District 4)](#oakland), [County Express (District 5)](#san-luis-obispo), [Delano (District 6)](#fresno), [DowneyLINK GMV (District 7)](#los-angeles), [Beaumont Pass (District 8)](#san-bernardino), [Tracy (District 10)](#stockton), and [Imperial Valley Transit (District 11)](#san-diego).


### Steady 24 Hour Service
* [Bay Area 511 Muni (District 4)](#oakland) provides some level of service at all hours of the day. While it drops to roughly 10-15% capacity between 1 AM - 4 AM, it's impressive that there are still over 12,000 stops with scheduled service on the system. Service is simiar on the weekends as it is on weekdays.
* [Bay Area 511 BART (District 4)](#oakland) is similar to Muni. Two notable differences are that there is no BART service between 2 AM - 3 AM on weekdays (2 AM - 4 AM on weekends) and that weekend service begins one hour later in the morning.

### Commuter Service
* [Bay Area 511 Mission Bay and Bay Area 511 Commute.org (District 4)](#oakland) both serve commuter riders primarily, with pronounced AM and PM peak service, no scheduled midday arrivals during weekdays, and no weekend service.

### Primarily Midday Service
* [Cerritos on Wheels (District 7)](#los-angeles) delivers meals to seniors, and operates strictly during business hours (9 AM - 5 PM). Operating strictly within typical business hours is consistent with an operator providing a type of government service. 

### Traveler Service
* [LAX FlyAway (District 7)](#los-angeles) operates round-the-clock, dipping only between 2 AM - 3 AM.
* [Anaheim Resort (District 12)](#irvine) continuously provides shuttle service to Disneyland visitors, except from 2 AM - 4 AM.

### Methodology

**About**

The Caltrans Division of Data & Digital Services (you may also know us as the [California Integrated Travel Project](https://www.calitp.org/)) has a data warehouse that daily ingests all the available GTFS schedule and real-time feeds for all transit operators in the state (and some outside of it too, like Amtrak)! We are working on modernize the tranpsortation experience through our data and digital lens. 

**Who / What / Where / When / Why / How**

**Who**: All California operators.

**What**: The main dataset used for this analysis is *stop times*, essentially one timetable that shows all the scheduled stop arrivals on a given day. 

**Where**: California - yes, this is for the entire state!

**When**: We are using the stop times tables spanning April 15 - 21, 2024.

**Why**: We can see what service patterns look like across operators by hour of the day. In one table, we can quickly see a summary snapshot of what this service pattern is and see the type of riders the operator typically serves. Here's the same [chart](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7160583/figure/fig6/) made in a [paper](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7160583) about for transit in Calgary, Canada.

**How**: All the stop times are binned by departure hour and aggregated by hour, day, and operator. There is a step to inject rows to hold zeroes where there is no scheduled stop arrival (since those would naturally *not* appear in our dataset). The dataset is then aggregated for each operator by weekday and weekend. To make regional comparisons easier, a crosswalk is merged in so we know which Caltrans District the operator belongs in.

## District Tables

In [ ]:
#for d in all_districts:
    #table = make_plot(df.filter(pl.col("caltrans_district") == d))
    #display(
    #    Markdown(f"### District {d}")
    #)
    #display(table)


### 01 - Eureka

In [ ]:
d = "01 - Eureka"
make_plot(df.filter(pl.col("caltrans_district") == d))

### 02 - Redding

In [ ]:
d = "02 - Redding"
make_plot(df.filter(pl.col("caltrans_district") == d))

### 03 - Marysville

In [ ]:
d = "03 - Marysville"
make_plot(df.filter(pl.col("caltrans_district") == d))

### 04 - Oakland

In [ ]:
d = "04 - Oakland"
make_plot(df.filter(pl.col("caltrans_district") == d))

### 05 - San Luis Obispo

In [ ]:
d = "05 - San Luis Obispo"
make_plot(df.filter(pl.col("caltrans_district") == d))

### 06 - Fresno

In [ ]:
d = "06 - Fresno"
make_plot(df.filter(pl.col("caltrans_district") == d))

### 07 - Los Angeles

In [ ]:
d = "07 - Los Angeles"
make_plot(df.filter(pl.col("caltrans_district") == d))

### 08 - San Bernardino

In [ ]:
d = "08 - San Bernardino"
make_plot(df.filter(pl.col("caltrans_district") == d))

### 09 - Bishop

In [ ]:
d = "09 - Bishop"
make_plot(df.filter(pl.col("caltrans_district") == d))

### 10 - Stockton

In [ ]:
d = "10 - Stockton"
make_plot(df.filter(pl.col("caltrans_district") == d))

### 11 - San Diego

In [ ]:
d = "11 - San Diego"
make_plot(df.filter(pl.col("caltrans_district") == d))

### 12 - Irvine

In [ ]:
d = "12 - Irvine"
make_plot(df.filter(pl.col("caltrans_district") == d))